In [1]:
import os
import json
from models.TFT.TFT_predictor import TFT_Predictor

In [2]:
TFT_pred = TFT_Predictor(model_group_name='test',load=False)

c:\Users\mal84\Repositories\Annex_37\models\TFT\TFT_predictor.py:106: UserWarning: Warning: A log directory for this model group name (test) already exists but you are not loading it.
  warnings.warn(f"Warning: A log directory for this model group name ({model_group_name}) already exists but you are not loading it.")
c:\Users\mal84\Repositories\Annex_37\models\TFT\TFT_predictor.py:110: UserWarning: Warning: Model group metadata overwritten.
  if os.path.exists(group_mparams_path): warnings.warn("Warning: Model group metadata overwritten.")
Global seed set to 42


In [3]:
train_path = os.path.join('data','example','train')
val_path = os.path.join('data','example','validate')
with open(os.path.join(train_path,'metadata_ext.json')) as json_file:
    UCam_ids = json.load(json_file)["UCam_building_ids"]

In [4]:
model_type = 'load'
model_name = 'load_test'
i = 0
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

c:\Users\mal84\Repositories\Annex_37\models\TFT\TFT_predictor.py:262: UserWarning: Warning: A logs directory already exists for the model name `load_test`. By continuing you will overwrite this model.
  warnings.warn(f"Warning: A logs directory already exists for the model name `{model_name}`. By continuing you will overwrite this model.")
c:\Users\mal84\.conda\envs\Annex37\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
c:\Users\mal84\.conda\envs\Annex37\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: False, used: Fa

TypeError: `model` must be a `LightningModule`, got `TemporalFusionTransformer`

In [ ]:
# create and train load models
model_type = 'load'
for i,id in enumerate(UCam_ids):
    model_name = f'{model_type}_{id}'
    train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
    tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
    TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train solar models
model_type = 'solar'
for i,id in enumerate(UCam_ids):
    model_name = f'{model_type}_{id}'
    train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
    tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
    TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train pricing model
model_type = 'pricing'
model_name = 'price'
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train carbon model
model_type = 'carbon'
model_name = 'carbon'
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
from citylearn.citylearn import CityLearnEnv
env = CityLearnEnv(os.path.join(val_path,'schema.json'))
t = 7720